# データサイエンス100本ノック（構造化データ加工編） - Python

### 独自にライブラリを追加

In [1]:
!pip install MonthDelta

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler
from monthdelta import monthmod
import re

if 'PG_PORT' in os.environ:
    pgconfig = {
        'host': 'db',
        'port': os.environ['PG_PORT'],
        'database': os.environ['PG_DATABASE'],
        'user': os.environ['PG_USER'],
        'password': os.environ['PG_PASSWORD'],
    }


    # pd.read_sql用のコネクタ
    conn = psycopg2.connect(**pgconfig)

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("data/category.csv", dtype=dtype)
    df_product = pd.read_csv("data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("data/geocode.csv", dtype=dtype)

/tmp/ipykernel_2429/2238586528.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql(sql='select * from customer', con=conn)
/tmp/ipykernel_2429/2238586528.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_category = pd.read_sql(sql='select * from category', con=conn)
/tmp/ipykernel_2429/2238586528.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_product = pd.read_sql(sql='select * from product', con=conn)
/tmp/ipykernel_2429/2238586528.py:33: UserWarning: pandas only supports SQLAlchem

# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [3]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [4]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdsales_dateに項目名を変更しながら抽出すること。

In [5]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns = {'sales_ymd': 'sales_date'}).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [6]:
df_receipt[df_receipt['customer_id'] == 'CS018205000001'][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [7]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [8]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & ((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5))][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [9]:
# df[df[～]]の範囲抽出ではbetweenに該当する機能なし
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & ((df_receipt['amount'] >= 1000) & (df_receipt['amount'] <= 2000))][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [10]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['product_cd'] != 'P071401019')][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [11]:
df_store.query('prefecture_cd != "13" & floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [12]:
# strによるトリミングは開始位置と終了位置の指定に注意
# 参考：https://pg-chain.com/python-str-substring#toc3
df_store[df_store['store_cd'].str[:3] == 'S14'].head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [13]:
df_customer[df_customer['customer_id'].str[-1:] == '1'].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [14]:
df_store[df_store['address'].str.contains('横浜市')]

# 正規表現でも抽出可能
# df_store[df_store['address'].str.match('.*横浜市.*')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [15]:
df_customer[df_customer['status_cd'].str.match('^[A-F].*')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [16]:
df_customer[df_customer['status_cd'].str.match('.*[1-9]$')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [17]:
df_customer[df_customer['status_cd'].str.match('^[A-F].*[1-9]$')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [18]:
df_store[df_store['tel_no'].str.match('^\d{3}-\d{3}-\d{4}$')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [19]:
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [20]:
df_customer.sort_values('birth_day', ascending = False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [21]:
df_019 = df_receipt[['customer_id', 'amount']].copy()
df_019['amount_rank'] = df_019['amount'].rank(method = 'min', ascending = False)

df_019.sort_values('amount_rank').head(10)

,customer_id,amount,amount_rank
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [22]:
df_020 = df_receipt[['customer_id', 'amount']].copy()
df_020['amount_rank'] = df_020['amount'].rank(method = 'first', ascending = False)

df_020.sort_values('amount_rank').head(10)

,customer_id,amount,amount_rank
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [23]:
len(df_receipt)

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [24]:
len(df_receipt['customer_id'].unique())

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [25]:
df_receipt[['store_cd', 'amount', 'quantity']].groupby('store_cd').agg(sum_amount = ('amount', 'sum'), sum_quantity = ('quantity', 'sum')).reset_index()

,store_cd,sum_amount,sum_quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [26]:
df_receipt[['customer_id', 'sales_ymd']].groupby('customer_id').agg(max_sales_ymd = ('sales_ymd', 'max')).reset_index().head(10)

,customer_id,max_sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [27]:
df_receipt[['customer_id', 'sales_ymd']].groupby('customer_id').agg(min_sales_ymd = ('sales_ymd', 'min')).reset_index().head(10)

,customer_id,min_sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [28]:
df_026 = df_receipt[['customer_id', 'sales_ymd']].copy()
df_026_grouped = df_026.groupby('customer_id').agg(max_sales_ymd = ('sales_ymd', 'max'), min_sales_ymd = ('sales_ymd', 'min')).reset_index()

df_026_grouped[df_026_grouped['max_sales_ymd'] != df_026_grouped['min_sales_ymd']].head(10)

,customer_id,max_sales_ymd,min_sales_ymd
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [29]:
df_027 = df_receipt[['store_cd', 'amount']].copy()
df_027_grouped = df_027.groupby('store_cd').agg(mean_amount = ('amount', 'mean')).reset_index()

df_027_grouped.sort_values('mean_amount', ascending = False).head(5)

,store_cd,mean_amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [30]:
df_028 = df_receipt[['store_cd', 'amount']].copy()
df_028_grouped = df_028.groupby('store_cd').agg(median_amount = ('amount', 'median')).reset_index()

df_028_grouped.sort_values('median_amount', ascending = False).head(5)

,store_cd,median_amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [31]:
# GroupByで標準提供されていない統計量を求める場合は、df.groupby('集約キー')['集計対象'].apply(lambda x: 統計量算出処理)
df_029 = df_receipt[['store_cd', 'product_cd']].copy()
df_029_grouped = df_029.groupby('store_cd')['product_cd'].apply(lambda x: x.mode()).reset_index()
df_029_grouped.columns = ['store_cd', 'mode', 'product_cd']

df_029_grouped.head(10)

,store_cd,mode,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [32]:
# 標本分散
df_030 = df_receipt[['store_cd', 'amount']].copy()
df_030_grouped = df_030.groupby('store_cd')['amount'].apply(lambda x: x.var(ddof = 0)).reset_index()
df_030_grouped.columns = ['store_cd', 'var_amount']

df_030_grouped.sort_values('var_amount', ascending = False).head(5)

,store_cd,var_amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [33]:
# 標本標準偏差
df_031 = df_receipt[['store_cd', 'amount']]
df_031_grouped = df_031.groupby('store_cd')['amount'].apply(lambda x: x.std(ddof = 0)).reset_index()
df_031_grouped.columns = ['store_cd', 'std_amount']

df_031_grouped.sort_values('std_amount', ascending = False).head(5)

,store_cd,std_amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [34]:
df_receipt['amount'].quantile([0.25, 0.5, 0.75, 1.0])

0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [35]:
df_033 = df_receipt[['store_cd', 'amount']].copy()
df_033_grouped = df_033.groupby('store_cd').agg(mean_amount = ('amount', 'mean')).reset_index()

df_033_grouped[df_033_grouped['mean_amount'] >= 330]

,store_cd,mean_amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [36]:
df_034 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_034_grouped = df_034.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_034_grouped['sum_amount'].mean()

2547.742234529256

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [37]:
df_035 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_035_grouped = df_035.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
mean_sum_amount = df_035_grouped['sum_amount'].mean()

df_035_grouped[df_035_grouped['sum_amount'] >= mean_sum_amount].head(10)

,customer_id,sum_amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [38]:
pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on = 'store_cd', how = 'inner').head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [39]:
pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], on = 'category_small_cd', how = 'inner').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [40]:
df_038_c = df_customer[(df_customer['gender_cd'] == '1') & (df_customer['customer_id'].str[:1] != 'Z')][['customer_id']].copy()
df_038_r = df_receipt[['customer_id', 'amount']].copy()

df_038 = pd.merge(df_038_c, df_038_r, on = 'customer_id', how = 'left').fillna(0)
df_038_grouped = df_038.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_038_grouped.head(10)

,customer_id,sum_amount
0,CS001112000009,0.0
1,CS001112000019,0.0
2,CS001112000021,0.0
3,CS001112000023,0.0
4,CS001112000024,0.0
5,CS001112000029,0.0
6,CS001112000030,0.0
7,CS001113000004,1298.0
8,CS001113000010,0.0
9,CS001114000005,626.0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [41]:
df_039 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'].copy()

df_039_grouped_sum_amount = df_039.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_039_grouped_sum_amount = df_039_grouped_sum_amount.sort_values('sum_amount', ascending = False).head(20)

# 売上日数の多い顧客を算出するために、cutomer_idとsales_ymdで一意にしたうえで集約する
df_039_unique = df_039[['customer_id', 'sales_ymd']].drop_duplicates().copy()
df_039_grouped_count_sales = df_039_unique.groupby('customer_id').agg(count_sales_ymd = ('sales_ymd', 'count')).reset_index()
df_039_grouped_count_sales = df_039_grouped_count_sales.sort_values('count_sales_ymd', ascending = False).head(20)

pd.merge(df_039_grouped_sum_amount, df_039_grouped_count_sales, on = 'customer_id', how = 'outer').sort_values('customer_id')

,customer_id,sum_amount,count_sales_ymd
4,CS001605000009,18925.0,NaN
7,CS006515000023,18372.0,NaN
16,CS007514000094,15735.0,NaN
17,CS009414000059,15492.0,NaN
21,CS010214000002,NaN,21.0
5,CS010214000010,18585.0,22.0
8,CS011414000106,18338.0,NaN
14,CS011415000006,16094.0,NaN
22,CS014214000023,NaN,19.0
30,CS014415000077,NaN,18.0


---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [42]:
# pandasのmerge（join）にはクロス結合を直接行えない
# 全レコードに共通キーを持たせ、full outer joinする
def cross_join(df_x, df_y):
    
    df_x['tmp'] = 1
    df_y['tmp'] = 1
    
    return pd.merge(df_x, df_y, how = 'outer')

len(cross_join(df_store, df_product))

531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [43]:
df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').agg(sum_amount = ('amount', 'sum')) \
    .sort_values('sales_ymd').diff().head(10)

,sum_amount
sales_ymd,
20170101,NaN
20170102,-9558.0
20170103,3338.0
20170104,8662.0
20170105,1665.0
20170106,-5443.0
20170107,-8972.0
20170108,1322.0
20170109,1981.0


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [44]:
df_042 = df_receipt[['sales_ymd', 'amount']]
df_042_grouped = df_042.groupby('sales_ymd').agg(sum_amount = ('amount', 'sum')).sort_values('sales_ymd').reset_index()

for index in range(3, len(df_042_grouped)):
    for lag in range(1,4):
        col = 'lag' + str(lag) + '_sum_amount'
        df_042_grouped.loc[index, [col]] = df_042_grouped.at[index - lag, 'sum_amount']

df_042_grouped.dropna().head(10)

,sales_ymd,sum_amount,lag1_sum_amount,lag2_sum_amount,lag3_sum_amount
3,20170104,36165,27503.0,24165.0,33723.0
4,20170105,37830,36165.0,27503.0,24165.0
5,20170106,32387,37830.0,36165.0,27503.0
6,20170107,23415,32387.0,37830.0,36165.0
7,20170108,24737,23415.0,32387.0,37830.0
8,20170109,26718,24737.0,23415.0,32387.0
9,20170110,20143,26718.0,24737.0,23415.0
10,20170111,24287,20143.0,26718.0,24737.0
11,20170112,23526,24287.0,20143.0,26718.0
12,20170113,28004,23526.0,24287.0,20143.0


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [45]:
df_043_c = df_customer[['customer_id', 'gender', 'age']].copy()
df_043_c['era'] = df_043_c['age'].apply(lambda x: '10代未満' if x < 10 else str(int(x / 10)) + '0代')
df_043_c.drop('age', axis = 1, inplace = True)

df_043_r = df_receipt[['customer_id', 'amount']].copy()

df_043 = pd.merge(df_043_c, df_043_r, on = 'customer_id', how = 'inner')

pd.pivot_table(df_043, index = 'era', columns = 'gender', aggfunc = np.sum)

/tmp/ipykernel_2429/3826166906.py:9: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  pd.pivot_table(df_043, index = 'era', columns = 'gender', aggfunc = np.sum)


amount                     
gender        不明         女性        男性
era                                  
10代       4317.0   149836.0    1591.0
20代      44328.0  1363724.0   72940.0
30代      50441.0   693047.0  177322.0
40代     483512.0  9320791.0   19355.0
50代     342923.0  6685192.0   54320.0
60代      71418.0   987741.0  272469.0
70代       2427.0    29764.0   13435.0
80代       5111.0   262923.0   46360.0
90代          NaN     6260.0       NaN

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [46]:
df_044_c = df_customer[['customer_id', 'gender', 'age']].copy()
df_044_c['era'] = df_044_c['age'].apply(lambda x: '10代未満' if x < 10 else str(int(x / 10)) + '0代')
df_044_c.drop('age', axis = 1, inplace = True)

df_044_c['gender'].replace(['東京', '女性', '不明'], ['00', '01', '99'])

df_044_r = df_receipt[['customer_id', 'amount']].copy()

df_044 = pd.merge(df_044_c, df_044_r, on = 'customer_id', how = 'inner')

df_044.groupby(['gender', 'era']).agg(sum_amount = ('amount', 'sum')).reset_index()

,gender,era,sum_amount
0,不明,10代,4317
1,不明,20代,44328
2,不明,30代,50441
3,不明,40代,483512
4,不明,50代,342923
5,不明,60代,71418
6,不明,70代,2427
7,不明,80代,5111
8,女性,10代,149836
9,女性,20代,1363724


---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [47]:
df_045 = df_customer[['customer_id', 'birth_day']].copy()
df_045.loc[:, 'birth_day'] = df_045['birth_day'].astype(str).str.replace('-', '')
df_045.head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [48]:
df_046 = df_customer[['customer_id', 'application_date']].copy()
df_046.loc[:, 'application_date'] = pd.to_datetime(df_046['application_date'])
df_046.head(10)

/tmp/ipykernel_2429/1791146345.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_046.loc[:, 'application_date'] = pd.to_datetime(df_046['application_date'])


,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [49]:
df_047 = df_receipt[['sales_ymd', 'receipt_no', 'receipt_sub_no']].copy()
df_047.loc[:, 'sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
df_047.head(10)

,sales_ymd,receipt_no,receipt_sub_no
0,2018-11-03,112,1
1,2018-11-18,1132,2
2,2017-07-12,1102,1
3,2019-02-05,1132,1
4,2018-08-21,1102,2
5,2019-06-05,1112,1
6,2018-12-05,1102,2
7,2019-09-22,1102,1
8,2017-05-04,1112,2
9,2019-10-10,1102,1


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [50]:
df_048 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_048['convert_sales_epoch'] = df_048['sales_epoch'].apply(datetime.fromtimestamp)
df_048.head(10)

,sales_epoch,receipt_no,receipt_sub_no,convert_sales_epoch
0,1541203200,112,1,2018-11-03
1,1542499200,1132,2,2018-11-18
2,1499817600,1102,1,2017-07-12
3,1549324800,1132,1,2019-02-05
4,1534809600,1102,2,2018-08-21
5,1559692800,1112,1,2019-06-05
6,1543968000,1102,2,2018-12-05
7,1569110400,1102,1,2019-09-22
8,1493856000,1112,2,2017-05-04
9,1570665600,1102,1,2019-10-10


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [51]:
df_049 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_049['convert_sales_epoch_y'] = df_049['sales_epoch'].apply(datetime.fromtimestamp).dt.year
df_049[['receipt_no', 'receipt_sub_no', 'convert_sales_epoch_y']].head(10)

,receipt_no,receipt_sub_no,convert_sales_epoch_y
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [52]:
df_050 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_050['convert_sales_epoch_m'] = df_050['sales_epoch'].apply(datetime.fromtimestamp).dt.month.astype(str).str.zfill(2)
df_050[['convert_sales_epoch_m', 'receipt_no', 'receipt_sub_no']].head(10)

,convert_sales_epoch_m,receipt_no,receipt_sub_no
0,11,112,1
1,11,1132,2
2,07,1102,1
3,02,1132,1
4,08,1102,2
5,06,1112,1
6,12,1102,2
7,09,1102,1
8,05,1112,2
9,10,1102,1


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [53]:
df_050 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_050['convert_sales_epoch_d'] = df_050['sales_epoch'].apply(datetime.fromtimestamp).dt.day.astype(str).str.zfill(2)
df_050[['convert_sales_epoch_d', 'receipt_no', 'receipt_sub_no']].head(10)

,convert_sales_epoch_d,receipt_no,receipt_sub_no
0,03,112,1
1,18,1132,2
2,12,1102,1
3,05,1132,1
4,21,1102,2
5,05,1112,1
6,05,1102,2
7,22,1102,1
8,04,1112,2
9,10,1102,1


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [54]:
df_052 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_052_grouped = df_052.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_052_grouped['sum_amount > 2000'] = df_052_grouped['sum_amount'].apply(lambda x: 1 if x > 2000 else 0)
df_052_grouped.head(10)

,customer_id,sum_amount,sum_amount > 2000
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [55]:
df_053_c = df_customer[['customer_id', 'postal_cd']].copy()
df_053_c['tokyo_flag'] = df_053_c['postal_cd'].str[:3].astype(int).apply(lambda x: 1 if 100 <= x <= 209 else 0)

df_053_r = df_receipt['customer_id'].drop_duplicates().copy()

df_053 = pd.merge(df_053_c, df_053_r, on = 'customer_id', how = 'inner')
df_053.groupby('tokyo_flag').agg(count_customer_id = ('customer_id', 'count')).reset_index()

,tokyo_flag,count_customer_id
0,0,3906
1,1,4400


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [56]:
pattern = '東京都|北海道|(京都|大阪)府|.{2,3}県'
df_054 = df_customer[['address']].copy()
df_054['address_code'] = df_054['address'].apply(lambda x: re.match(pattern, x).group()).replace(['埼玉県', '千葉県', '東京都', '神奈川県'], [11, 12, 13, 14])
df_054.head(10)

,address,address_code
0,神奈川県伊勢原市粟窪**********,14
1,東京都江東区南砂**********,13
2,東京都渋谷区代々木**********,13
3,神奈川県横浜市泉区和泉町**********,14
4,東京都大田区仲六郷**********,13
5,東京都板橋区若木**********,13
6,東京都江東区北砂**********,13
7,千葉県浦安市海楽**********,12
8,東京都江東区北砂**********,13
9,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [57]:
def quantile_code(x):
    
    if x < sum_amount_quantile[0.25]:
        return 1
    elif sum_amount_quantile[0.25] <= x < sum_amount_quantile[0.5]:
        return 2
    elif sum_amount_quantile[0.5] <= x < sum_amount_quantile[0.75]:
        return 3
    else:
        return 4   

df_055 = df_receipt[['customer_id', 'amount']].copy()
df_055_grouped = df_055.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()

sum_amount_quantile = df_055_grouped['sum_amount'].quantile([0.25, 0.5, 0.75, 1.0])
 
df_055_grouped['quantile_code'] = df_055_grouped['sum_amount'].apply(quantile_code)

df_055_grouped.head(10)

,customer_id,sum_amount,quantile_code
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [58]:
df_056 = df_customer[['customer_id', 'age', 'birth_day']].copy()
df_056['era'] = df_056['age'].apply(lambda x: '10代未満' if x < 10 else ('60代' if x > 60 else str(int(x / 10)) + '0代'))
df_056[['customer_id', 'birth_day', 'era']].head(10)

,customer_id,birth_day,era
0,CS021313000114,1981-04-29,30代
1,CS037613000071,1952-04-01,60代
2,CS031415000172,1976-10-04,40代
3,CS028811000001,1933-03-27,60代
4,CS001215000145,1995-03-29,20代
5,CS020401000016,1974-09-15,40代
6,CS015414000103,1977-08-09,40代
7,CS029403000008,1973-08-17,40代
8,CS015804000004,1931-05-02,60代
9,CS033513000180,1962-07-11,50代


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [59]:
df_057 = df_customer[['customer_id', 'age', 'birth_day', 'gender']].copy()
df_057['era'] = df_057['age'].apply(lambda x: '10代未満' if x < 10 else ('60代' if x > 60 else str(int(x / 10)) + '0代'))
df_057['era + gender'] = df_057['era'] + df_057['gender']
df_057.head(10)

,customer_id,age,birth_day,gender,era,era + gender
0,CS021313000114,37,1981-04-29,女性,30代,30代女性
1,CS037613000071,66,1952-04-01,不明,60代,60代不明
2,CS031415000172,42,1976-10-04,女性,40代,40代女性
3,CS028811000001,86,1933-03-27,女性,60代,60代女性
4,CS001215000145,24,1995-03-29,女性,20代,20代女性
5,CS020401000016,44,1974-09-15,男性,40代,40代男性
6,CS015414000103,41,1977-08-09,女性,40代,40代女性
7,CS029403000008,45,1973-08-17,男性,40代,40代男性
8,CS015804000004,87,1931-05-02,男性,60代,60代男性
9,CS033513000180,56,1962-07-11,女性,50代,50代女性


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [60]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [61]:
stdsc = preprocessing.StandardScaler()
df_059 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_059_grouped = df_059.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_059_grouped['std_amount'] = stdsc.fit_transform(df_059_grouped[['sum_amount']].values)
df_059_grouped.head(10)

,sum_amount,std_amount
customer_id,,
CS001113000004,1298,-0.459378
CS001114000005,626,-0.706390
CS001115000010,3044,0.182413
CS001205000004,1988,-0.205749
CS001205000006,3337,0.290114
CS001211000025,456,-0.768879
CS001212000027,448,-0.771819
CS001212000031,296,-0.827691
CS001212000046,228,-0.852686


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [62]:
mmsc = preprocessing.MinMaxScaler()
df_060 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_060_grouped = df_060.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_060_grouped['mm_amount'] = mmsc.fit_transform(df_060_grouped[['sum_amount']].values)
df_060_grouped.head(10)

,sum_amount,mm_amount
customer_id,,
CS001113000004,1298,0.053354
CS001114000005,626,0.024157
CS001115000010,3044,0.129214
CS001205000004,1988,0.083333
CS001205000006,3337,0.141945
CS001211000025,456,0.016771
CS001212000027,448,0.016423
CS001212000031,296,0.009819
CS001212000046,228,0.006865


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [63]:
df_061 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_061_grouped = df_061.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_061_grouped['log10_amount'] = np.log10(df_061_grouped['sum_amount'])

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [64]:
df_062 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_062_grouped = df_062.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_062_grouped['log_amount'] = np.log(df_062_grouped['sum_amount'])

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [65]:
df_063 = df_product.copy()
df_063['profit'] = df_063['unit_price'] - df_063['unit_cost']
df_063.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,profit
0,P040101001,04,0401,040101,198.0,149.0,1,49.0
1,P040101002,04,0401,040101,218.0,164.0,1,54.0
2,P040101003,04,0401,040101,230.0,173.0,1,57.0
3,P040101004,04,0401,040101,248.0,186.0,1,62.0
4,P040101005,04,0401,040101,268.0,201.0,1,67.0
5,P040101006,04,0401,040101,298.0,224.0,1,74.0
6,P040101007,04,0401,040101,338.0,254.0,1,84.0
7,P040101008,04,0401,040101,420.0,315.0,1,105.0
8,P040101009,04,0401,040101,498.0,374.0,1,124.0
9,P040101010,04,0401,040101,580.0,435.0,1,145.0


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [66]:
df_064 = df_product.copy()

# 本問題では利益率 = (単価 - 原価) / 単価で算出
df_064['profit_rate'] = (df_064['unit_price'] - df_064['unit_cost']) / df_064['unit_price']
df_064['profit_rate'].mean()

0.24911389885177

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [67]:
df_065 = df_product.copy()
df_065['pr30_unit_price'] = (df_065['unit_cost'] / 0.7).agg(np.floor)
df_065.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,pr30_unit_price
0,P040101001,04,0401,040101,198.0,149.0,1,212.0
1,P040101002,04,0401,040101,218.0,164.0,1,234.0
2,P040101003,04,0401,040101,230.0,173.0,1,247.0
3,P040101004,04,0401,040101,248.0,186.0,1,265.0
4,P040101005,04,0401,040101,268.0,201.0,1,287.0
5,P040101006,04,0401,040101,298.0,224.0,1,320.0
6,P040101007,04,0401,040101,338.0,254.0,1,362.0
7,P040101008,04,0401,040101,420.0,315.0,1,450.0
8,P040101009,04,0401,040101,498.0,374.0,1,534.0
9,P040101010,04,0401,040101,580.0,435.0,1,621.0


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [68]:
df_066 = df_product.copy()
df_066['pr30_unit_price'] = (df_066['unit_cost'] / 0.7).round()
df_066.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,pr30_unit_price
0,P040101001,04,0401,040101,198.0,149.0,1,213.0
1,P040101002,04,0401,040101,218.0,164.0,1,234.0
2,P040101003,04,0401,040101,230.0,173.0,1,247.0
3,P040101004,04,0401,040101,248.0,186.0,1,266.0
4,P040101005,04,0401,040101,268.0,201.0,1,287.0
5,P040101006,04,0401,040101,298.0,224.0,1,320.0
6,P040101007,04,0401,040101,338.0,254.0,1,363.0
7,P040101008,04,0401,040101,420.0,315.0,1,450.0
8,P040101009,04,0401,040101,498.0,374.0,1,534.0
9,P040101010,04,0401,040101,580.0,435.0,1,621.0


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [69]:
df_067 = df_product.copy()
df_067['pr30_unit_price'] = (df_067['unit_cost'] / 0.7).agg(np.ceil)
df_067.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,pr30_unit_price
0,P040101001,04,0401,040101,198.0,149.0,1,213.0
1,P040101002,04,0401,040101,218.0,164.0,1,235.0
2,P040101003,04,0401,040101,230.0,173.0,1,248.0
3,P040101004,04,0401,040101,248.0,186.0,1,266.0
4,P040101005,04,0401,040101,268.0,201.0,1,288.0
5,P040101006,04,0401,040101,298.0,224.0,1,320.0
6,P040101007,04,0401,040101,338.0,254.0,1,363.0
7,P040101008,04,0401,040101,420.0,315.0,1,451.0
8,P040101009,04,0401,040101,498.0,374.0,1,535.0
9,P040101010,04,0401,040101,580.0,435.0,1,622.0


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [70]:
df_068 = df_product.copy()
df_068['tax_included_unit_price'] = (df_068['unit_price'] * 1.1).agg(np.floor)
df_068.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,tax_included_unit_price
0,P040101001,04,0401,040101,198.0,149.0,1,217.0
1,P040101002,04,0401,040101,218.0,164.0,1,239.0
2,P040101003,04,0401,040101,230.0,173.0,1,253.0
3,P040101004,04,0401,040101,248.0,186.0,1,272.0
4,P040101005,04,0401,040101,268.0,201.0,1,294.0
5,P040101006,04,0401,040101,298.0,224.0,1,327.0
6,P040101007,04,0401,040101,338.0,254.0,1,371.0
7,P040101008,04,0401,040101,420.0,315.0,1,462.0
8,P040101009,04,0401,040101,498.0,374.0,1,547.0
9,P040101010,04,0401,040101,580.0,435.0,1,638.0


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [71]:
df_069 = pd.merge(df_receipt, df_product, on = 'product_cd', how = 'inner')

df_069_grouped_tmp1 = df_069.groupby('customer_id').agg(sum_all_amount = ('amount', 'sum')).reset_index()

df_069_tmp2 = df_069[df_069['category_major_cd'] == '07'].copy()
df_069_grouped_tmp2 = df_069_tmp2.groupby('customer_id').agg(sum_07_amount = ('amount', 'sum')).reset_index()

df_069 = pd.merge(df_069_grouped_tmp1, df_069_grouped_tmp2, on = 'customer_id', how = 'right')
df_069['rate'] = df_069['sum_07_amount'] / df_069['sum_all_amount']
df_069.head(10)

,customer_id,sum_all_amount,sum_07_amount,rate
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [72]:
df_070 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_070['application_date'] = pd.to_datetime(df_070['application_date'])
df_070['sales_ymd'] = pd.to_datetime(df_070['sales_ymd'].astype(str))
df_070['date_diff'] = df_070['sales_ymd'] - df_070['application_date']
df_070[['customer_id', 'sales_ymd', 'application_date', 'date_diff']].head(10)

,customer_id,sales_ymd,application_date,date_diff
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
3,CS006214000001,2017-06-08,2015-02-01,858 days
4,CS006214000001,2018-10-28,2015-02-01,1365 days
5,CS006214000001,2018-10-28,2015-02-01,1365 days
6,CS006214000001,2017-05-09,2015-02-01,828 days
7,CS006214000001,2019-09-08,2015-02-01,1680 days
8,CS006214000001,2018-01-31,2015-02-01,1095 days
9,CS006214000001,2017-07-05,2015-02-01,885 days


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [73]:
def diff_month(row):
    return monthmod(row['application_date'], row['sales_ymd'])[0].months
    
df_071 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_071['application_date'] = pd.to_datetime(df_071['application_date'])
df_071['sales_ymd'] = pd.to_datetime(df_071['sales_ymd'].astype(str))
df_071['diff_month'] = df_071.apply(diff_month, axis = 1)
df_071[['customer_id', 'application_date', 'sales_ymd', 'diff_month']].head(10)

,customer_id,application_date,sales_ymd,diff_month
0,CS006214000001,2015-02-01,2018-11-03,45
1,CS006214000001,2015-02-01,2017-05-09,27
2,CS006214000001,2015-02-01,2017-06-08,28
3,CS006214000001,2015-02-01,2017-06-08,28
4,CS006214000001,2015-02-01,2018-10-28,44
5,CS006214000001,2015-02-01,2018-10-28,44
6,CS006214000001,2015-02-01,2017-05-09,27
7,CS006214000001,2015-02-01,2019-09-08,55
8,CS006214000001,2015-02-01,2018-01-31,35
9,CS006214000001,2015-02-01,2017-07-05,29


---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [74]:
def diff_year(row):
    return monthmod(row['application_date'], row['sales_ymd'])[0].months // 12
    
df_071 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_071['application_date'] = pd.to_datetime(df_071['application_date'])
df_071['sales_ymd'] = pd.to_datetime(df_071['sales_ymd'].astype(str))
df_071['diff_year'] = df_071.apply(diff_year, axis = 1)
df_071[['customer_id', 'application_date', 'sales_ymd', 'diff_year']].head(10)

,customer_id,application_date,sales_ymd,diff_year
0,CS006214000001,2015-02-01,2018-11-03,3
1,CS006214000001,2015-02-01,2017-05-09,2
2,CS006214000001,2015-02-01,2017-06-08,2
3,CS006214000001,2015-02-01,2017-06-08,2
4,CS006214000001,2015-02-01,2018-10-28,3
5,CS006214000001,2015-02-01,2018-10-28,3
6,CS006214000001,2015-02-01,2017-05-09,2
7,CS006214000001,2015-02-01,2019-09-08,4
8,CS006214000001,2015-02-01,2018-01-31,2
9,CS006214000001,2015-02-01,2017-07-05,2


---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [75]:
df_073 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_073['application_date'] = pd.to_datetime(df_073['application_date']).astype(int)
df_073['sales_ymd'] = pd.to_datetime(df_073['sales_ymd'].astype(str)).astype(int)
df_073['diff_epoch'] = (df_073['sales_ymd'] - df_073['application_date']) / 10 ** 9
df_073[['customer_id', 'application_date', 'sales_ymd', 'diff_epoch']].head(10)

,customer_id,application_date,sales_ymd,diff_epoch
0,CS006214000001,1422748800000000000,1541203200000000000,118454400.0
1,CS006214000001,1422748800000000000,1494288000000000000,71539200.0
2,CS006214000001,1422748800000000000,1496880000000000000,74131200.0
3,CS006214000001,1422748800000000000,1496880000000000000,74131200.0
4,CS006214000001,1422748800000000000,1540684800000000000,117936000.0
5,CS006214000001,1422748800000000000,1540684800000000000,117936000.0
6,CS006214000001,1422748800000000000,1494288000000000000,71539200.0
7,CS006214000001,1422748800000000000,1567900800000000000,145152000.0
8,CS006214000001,1422748800000000000,1517356800000000000,94608000.0
9,CS006214000001,1422748800000000000,1499212800000000000,76464000.0


---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [76]:
def last_monday_ymd(row):
    return row['sales_ymd'] - timedelta(days = row['elapsed_day'])

df_074 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_074['sales_ymd'] = pd.to_datetime(df_074['sales_ymd'].astype(str))
df_074['elapsed_day'] = df_074['sales_ymd'].apply(lambda x: x.weekday())
df_074['last_monday_ymd'] = df_074.apply(last_monday_ymd, axis = 1)
df_074[['sales_ymd', 'elapsed_day', 'last_monday_ymd']].head(10)

,sales_ymd,elapsed_day,last_monday_ymd
0,2018-11-03,5,2018-10-29
1,2017-05-09,1,2017-05-08
2,2017-06-08,3,2017-06-05
3,2017-06-08,3,2017-06-05
4,2018-10-28,6,2018-10-22
5,2018-10-28,6,2018-10-22
6,2017-05-09,1,2017-05-08
7,2019-09-08,6,2019-09-02
8,2018-01-31,2,2018-01-29
9,2017-07-05,2,2017-07-03


---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [77]:
df_customer.sample(frac = 0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
9905,CS018513000212,河原 南朋,1,女性,1966-04-22,52,204-0012,東京都清瀬市中清戸**********,S13018,20150608,8-20090323-9
12486,CS020712000047,石田 瞳,1,女性,1947-10-23,71,174-0071,東京都板橋区常盤台**********,S13020,20141210,0-00000000-0
5386,CS012412000002,柳田 くるみ,1,女性,1974-02-15,45,231-0023,神奈川県横浜市中区山下町**********,S14012,20150913,0-00000000-0
9699,CS030302000010,大山 明,0,男性,1985-10-04,33,273-0035,千葉県船橋市本中山**********,S12030,20141104,0-00000000-0
5539,CS026312000131,砂川 はるみ,1,女性,1985-11-14,33,253-0051,神奈川県茅ヶ崎市若松町**********,S14026,20150522,0-00000000-0
19091,CS001313000431,吉田 浩介,9,不明,1981-01-18,38,144-0054,東京都大田区新蒲田**********,S13001,20161010,0-00000000-0
4168,CS022613000123,早美 俊二,9,不明,1956-07-18,62,249-0005,神奈川県逗子市桜山**********,S14022,20180829,0-00000000-0
11350,CS038414000076,稲垣 あさみ,1,女性,1971-05-28,47,134-0085,東京都江戸川区南葛西**********,S13038,20160130,D-20100204-D
6546,CS004512000088,岩淵 花,1,女性,1963-05-06,55,176-0004,東京都練馬区小竹町**********,S13004,20151228,0-00000000-0
14874,CS025611000003,浜本 窈,1,女性,1951-02-14,68,252-0823,神奈川県藤沢市菖蒲沢**********,S14025,20141125,0-00000000-0


---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [78]:
train, test = train_test_split(df_customer, test_size=0.1, stratify = df_customer['gender_cd'])

test.groupby('gender_cd').agg(count_customer_id = ('customer_id', 'count')).reset_index()

,gender_cd,count_customer_id
0,0,298
1,1,1793
2,9,107


---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [79]:
df_077 = df_receipt[['customer_id', 'amount']].copy()
df_077_grouped = df_077.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_077_grouped['log_amount'] = np.log(df_077_grouped['sum_amount'])

mean_log_amount = df_077_grouped['log_amount'].mean()
std_log_amount = df_077_grouped['log_amount'].std()

lower = mean_log_amount - 3 * std_log_amount
upper = mean_log_amount + 3 * std_log_amount

df_077_grouped[(df_077_grouped['log_amount'] < lower) | (df_077_grouped['log_amount'] > upper)].head(10)

,customer_id,sum_amount,log_amount
8306,ZZ000000000000,12395003,16.332804


---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [80]:
df_078 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'].copy()
df_078_grouped = df_078.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()

q1 = df_078_grouped['sum_amount'].quantile(0.25)
q3 = df_078_grouped['sum_amount'].quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

df_078_grouped[(df_078_grouped['sum_amount'] < lower) | (df_078_grouped['sum_amount'] > upper)].head(10)

,customer_id,sum_amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [81]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
tmp                   0
dtype: int64

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [82]:
df_080 = df_product.dropna(how = 'any').copy()

print('削除前：' + str(len(df_product)))
print('削除後：' + str(len(df_080)))

削除前：10030
削除後：10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [83]:
df_081 = df_product.fillna(df_product.mean().round()).copy()
df_081.isnull().sum()

/tmp/ipykernel_2429/3473929063.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_081 = df_product.fillna(df_product.mean().round()).copy()


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
tmp                   0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [84]:
df_082 = df_product.fillna(df_product.median().round()).copy()
df_082.isnull().sum()

/tmp/ipykernel_2429/4086633506.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_082 = df_product.fillna(df_product.median().round()).copy()


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
tmp                   0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [85]:
df_083 = df_product.copy()
df_083_grouped = df_product.groupby('category_small_cd').agg(median_unit_price = ('unit_price', 'median'), median_unit_cost = ('unit_cost', 'median'))

for category_small_cd, median_unit_price in df_083_grouped['median_unit_price'].items():
    df_083.loc[(df_083['category_small_cd'] == category_small_cd) & (df_083['unit_price'].isnull()), 'unit_price'] = median_unit_price
for category_small_cd, median_unit_cost in df_083_grouped['median_unit_cost'].items():
    df_083.loc[(df_083['category_small_cd'] == category_small_cd) & (df_083['unit_cost'].isnull()), 'unit_cost'] = median_unit_cost

df_083.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
tmp                   0
dtype: int64

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [86]:
df_084_all_r = df_receipt[['customer_id', 'amount']].copy()
df_084_all_r_grouped = df_084_all_r.groupby('customer_id').agg(all_amount = ('amount', 'sum')).reset_index()

df_084_2019_r = df_receipt[df_receipt['sales_ymd'].astype(str).str[:4] == '2019'][['customer_id', 'amount']].copy()
df_084_2019_r_grouped = df_084_2019_r.groupby('customer_id').agg(y2019_amount = ('amount', 'sum')).reset_index()
   
df_084 = pd.merge(pd.merge(df_customer['customer_id'], df_084_all_r_grouped, on = 'customer_id', how = 'left'),
                  df_084_2019_r_grouped, on = 'customer_id', how = 'left')

df_084['rate'] = df_084['y2019_amount'] / df_084['all_amount']
df_084.fillna(0, inplace = True)

df_084.head(10)

,customer_id,all_amount,y2019_amount,rate
0,CS021313000114,0.0,0.0,0.000000
1,CS037613000071,0.0,0.0,0.000000
2,CS031415000172,5088.0,2971.0,0.583923
3,CS028811000001,0.0,0.0,0.000000
4,CS001215000145,875.0,0.0,0.000000
5,CS020401000016,0.0,0.0,0.000000
6,CS015414000103,3122.0,874.0,0.279949
7,CS029403000008,0.0,0.0,0.000000
8,CS015804000004,0.0,0.0,0.000000
9,CS033513000180,868.0,0.0,0.000000


---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [87]:
df_085_g = df_geocode[['postal_cd', 'longitude', 'latitude']].copy()
df_085_g_grouped = df_085_g.groupby('postal_cd').agg(mean_longitude = ('longitude', 'mean'), mean_latitude = ('latitude', 'mean')).reset_index()
df_085 = pd.merge(df_customer, df_085_g_grouped, on = 'postal_cd', how = 'inner')
df_085.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,mean_longitude,mean_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,0-00000000-0,139.31779,35.41358
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,0-00000000-0,139.31779,35.41358
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,0-00000000-0,139.31779,35.41358
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,0-00000000-0,139.31779,35.41358
5,CS021314000093,江藤 美佐,1,女性,1986-06-03,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20151015,0-00000000-0,139.31779,35.41358
6,CS021413000049,大野 幸子,1,女性,1973-04-17,45,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150703,0-00000000-0,139.31779,35.41358
7,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
8,CS015415000209,大谷 倫子,1,女性,1970-11-25,48,136-0076,東京都江東区南砂**********,S13015,20150909,B-20090610-C,139.83502,35.67193
9,CS037614000045,長沢 麻緒,1,女性,1952-06-19,66,136-0076,東京都江東区南砂**********,S13037,20150209,5-20091207-6,139.83502,35.67193


---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [88]:
def calc_dist(x1, x2, y1, y2):
    dist = 6371 * math.acos(math.sin(math.radians(x1)) 
                * math.sin(math.radians(x2)) 
                + math.cos(math.radians(x1)) 
                * math.cos(math.radians(x2)) 
                * math.cos(math.radians(y1) - math.radians(y2)))
    return dist

df_086_g = df_geocode[['postal_cd', 'longitude', 'latitude']].copy()
df_086_g_grouped = df_086_g.groupby('postal_cd').agg(mean_longitude = ('longitude', 'mean'), mean_latitude = ('latitude', 'mean')).reset_index()
df_086 = pd.merge(df_customer[['customer_id', 'postal_cd', 'address', 'application_store_cd']], df_086_g_grouped, on = 'postal_cd', how = 'inner')
df_086.rename(columns = {'postal_cd': 'home_postal_cd', 'address': 'home_address',
                         'application_store_cd': 'store_cd', 'mean_longitude': 'home_longitude', 'mean_latitude': 'home_latitude'},
              inplace = True)

df_086_s = df_store[['store_cd', 'address', 'longitude', 'latitude']].copy()
df_086_s.rename(columns = {'address': 'store_address', 'longitude': 'store_longitude', 'latitude': 'store_latitude'}, inplace = True)
df_086 = pd.merge(df_086, df_086_s, on = 'store_cd', how = 'inner')
df_086['dist'] = df_086.apply(lambda x: calc_dist(x['home_latitude'], x['store_latitude'], x['home_longitude'], x['store_longitude']), axis = 1)

df_086[['customer_id', 'home_address', 'store_address', 'dist']].head(10)

,customer_id,home_address,store_address,dist
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS021303000023,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
2,CS021303000007,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
3,CS021313000183,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
4,CS021314000098,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
5,CS021314000093,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
6,CS021413000049,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
7,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
8,CS021413000022,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
9,CS021413000094,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282


---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [89]:
df_087_c_duplication = pd.merge(df_customer,
                                df_customer[df_customer.duplicated(subset = ['customer_name', 'postal_cd'])][['customer_name', 'postal_cd']],
                                on = ['customer_name', 'postal_cd'], how = 'inner')
df_087_r_duplication = pd.merge(df_receipt, df_087_c_duplication[['customer_id', 'customer_name', 'postal_cd']], on = 'customer_id', how = 'right')
df_087_r_duplication_grouped = df_087_r_duplication.groupby(['customer_id', 'customer_name', 'postal_cd']).agg(sum_amount = ('amount', 'sum')).reset_index()
df_087_r_duplication_grouped.sort_values(['customer_name', 'postal_cd', 'sum_amount', 'customer_id'], ascending=[False , False, False, True], inplace = True)
df_087_c_identified = df_087_r_duplication_grouped.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first').copy()
df_087_c_identified['identified_flag'] = True

df_087 = pd.merge(df_customer, df_087_c_identified[['customer_id', 'identified_flag']], on = 'customer_id', how = 'left')
df_087.sort_values(['customer_name', 'postal_cd', 'identified_flag'], inplace = True)
df_087.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first', inplace = True)

print('顧客データ数：' + str(len(df_customer)))
print('名寄顧客データ数：' + str(len(df_087)))
print('重複数データ数：' + str(len(df_087_c_identified)))

顧客データ数：21971
名寄顧客データ数：21941
重複数データ数：30


---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [90]:
df_088_c_duplication = pd.merge(df_customer,
                                df_customer[df_customer.duplicated(subset = ['customer_name', 'postal_cd'])][['customer_name', 'postal_cd']],
                                on = ['customer_name', 'postal_cd'], how = 'inner')
df_088_r_duplication = pd.merge(df_receipt, df_088_c_duplication[['customer_id', 'customer_name', 'postal_cd']], on = 'customer_id', how = 'right')
df_088_r_duplication_grouped = df_088_r_duplication.groupby(['customer_id', 'customer_name', 'postal_cd']).agg(sum_amount = ('amount', 'sum')).reset_index()
df_088_r_duplication_grouped.sort_values(['customer_name', 'postal_cd', 'sum_amount', 'customer_id'], ascending=[False , False, False, True], inplace = True)
df_088_c_identified = df_088_r_duplication_grouped.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first').copy()
df_088_c_identified['identified_flag'] = True
df_088_c_identified['identified_customer_id'] = df_088_c_identified['customer_id']

df_088 = pd.merge(df_customer, df_088_c_identified[['customer_id', 'identified_flag', 'identified_customer_id']], on = 'customer_id', how = 'left')
df_088.sort_values(['customer_name', 'postal_cd', 'identified_flag'], inplace = True)

df_088['identified_customer_id'].fillna(df_088['customer_id'], inplace = True)

print('顧客IDのユニーク件数：' + str(df_088['customer_id'].nunique()))
print('統合名寄IDのユニーク件数：' + str(df_088['identified_customer_id'].nunique()))

顧客IDのユニーク件数：21971
統合名寄IDのユニーク件数：21971


---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [91]:
df_089_r = df_receipt['customer_id'].drop_duplicates().copy()
df_089 = pd.merge(df_customer, df_089_r, on = 'customer_id', how = 'inner')

df_train, df_test = train_test_split(df_089, test_size=0.2)

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [92]:
df_090 = df_receipt[['sales_ymd', 'amount']].copy()
df_090['sales_ym'] = df_receipt['sales_ymd'].astype(str).str[:6]
df_090_grouped = df_090.groupby('sales_ym').agg(sum_amount = ('amount', 'sum')).reset_index()

df_train = []
df_test = []

tss = TimeSeriesSplit(n_splits = 3, max_train_size = 12, test_size = 6)
for train, test in tss.split(df_090_grouped):
    df_train.append(df_090_grouped.loc[train, :])
    df_test.append(df_090_grouped.loc[test, :])

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [93]:
df_091_r = df_receipt.drop_duplicates(subset = 'customer_id').copy()
df_091_r['buy_flag'] = True

df_091 = pd.merge(df_customer, df_091_r[['customer_id', 'buy_flag']], on = 'customer_id', how = 'left')

df_091_buy = df_091[df_091['buy_flag'] == True]
df_091_not_buy = df_091[df_091['buy_flag'] != True].sample(len(df_091_buy))

print('売上実績有顧客：' + str(len(df_091_buy)))
print('売上実績無顧客：' + str(len(df_091_not_buy)))

売上実績有顧客：8306
売上実績無顧客：8306


---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [94]:
df_092_c = df_customer.drop(columns = 'gender')
df_092_g = df_customer[['gender_cd', 'gender']].drop_duplicates()

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [95]:
df_093_c = df_category[['category_major_name', 'category_medium_name', 'category_small_name', 'category_small_cd']]
df_093 = pd.merge(df_product, df_093_c, on = 'category_small_cd', how = 'inner')

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [96]:
df_093.to_csv('data/df_094.csv', encoding = 'utf-8', index = False)

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [97]:
df_093.to_csv('data/df_095.csv', encoding = 'cp932', index = False)

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [98]:
df_093.to_csv('data/df_096.csv', encoding = 'utf-8', index = False, header = False)

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [99]:
df_097 = pd.read_csv('data/df_094.csv', encoding = 'utf-8')
df_097.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,1,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,1,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,1,惣菜,御飯類,弁当類


---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [100]:
df_098 = pd.read_csv('data/df_096.csv', encoding = 'utf-8', header = None)
df_098.head(3)

,0,1,2,3,4,5,6,7,8,9
0,P040101001,4,401,40101,198.0,149.0,1,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,1,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,1,惣菜,御飯類,弁当類


---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [101]:
df_093.to_csv('data/df_099.tsv', encoding = 'utf-8', sep = '\t', index = False)

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [102]:
df_100 = pd.read_csv('data/df_099.tsv', encoding = 'utf-8', sep = '\t')
df_100.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tmp,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,1,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,1,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,1,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！